In [ ]:
import pandas as pd
import os
import shutil
from google.colab import drive
from pathlib import Path
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SAVE_CSV_PATH = '/content/drive/MyDrive/HECTO/Dataset/CSV/DF_final.csv'
ZIP_PATH = "/content/drive/MyDrive/HECTO/Dataset/final_files/DF_Frames_Final.zip"
LOCAL_EXTRACT_BASE = Path("/content/DF_Extract")

In [ ]:
if LOCAL_EXTRACT_BASE.exists():
    shutil.rmtree(LOCAL_EXTRACT_BASE)
LOCAL_EXTRACT_BASE.mkdir(parents=True, exist_ok=True)

print(f"🔓 압축 해제 시작: {ZIP_PATH}")
# -q: quiet, -d: target directory
!unzip -q "{ZIP_PATH}" -d "{LOCAL_EXTRACT_BASE}"
print("✅ 압축 해제 완료!")

DF_FRAMES_LOCAL = LOCAL_EXTRACT_BASE / "DF_frames"

data_list = []
print("🔍 로컬 폴더 스캔 중 (Real=1, Fake=0)...")

🔓 압축 해제 시작: /content/drive/MyDrive/HECTO/Dataset/final_files/DF_Frames_Final.zip
✅ 압축 해제 완료!
🔍 로컬 폴더 스캔 중 (Real=1, Fake=0)...


In [ ]:
all_images = list(LOCAL_EXTRACT_BASE.rglob("*.jpg"))

data_list = [] # 리스트 초기화 확인

for img_path in all_images:
    path_str = str(img_path).lower()
    parts = img_path.parts

    # 1. 라벨링
    label = 1 if 'real' in path_str else 0

    # 2. 조작 방식/레벨 추출 (if-else 문법 교정)
    if label == 0:
        # Fake인 경우 상위 폴더(mix_2 또는 level_3 등)를 가져옴
        manip_method = parts[-3]
    else:
        # Real인 경우 'original'로 표시
        manip_method = 'original'

    # 3. Video ID 및 Base ID 추출 (if-else 밖으로 빼서 공통 적용)
    # 이미지 바로 위 폴더명 (예: 000_M001)
    video_id = img_path.parent.name

    # 타겟 일치를 위한 앞 3자리 숫자 (예: 000)
    base_id = video_id[:3]

    data_list.append({
        'video_id': video_id,
        'frame_path': str(img_path),
        'label': label,
        'source': 'DF',
        'manipulation_method': manip_method,
        'base_id': base_id
    })

df = pd.DataFrame(data_list)

# --- 이후 분할 로직은 동일하게 진행 ---
# 2. Base ID 기준 8:1:1 랜덤 분할
unique_bases = pd.Series(df['base_id'].unique())
print(f"📊 발견된 고유 타겟(Base ID) 수: {len(unique_bases)}개")

train_b, temp_b = train_test_split(unique_bases, test_size=0.2, random_state=42)
val_b, test_b = train_test_split(temp_b, test_size=0.5, random_state=42)

# 3. Split 마킹
df['split'] = None
df.loc[df['base_id'].isin(train_b), 'split'] = 'train'
df.loc[df['base_id'].isin(val_b), 'split'] = 'val'
df.loc[df['base_id'].isin(test_b), 'split'] = 'test'

# 분할용 임시 칼럼 삭제
df_final = df.drop(columns=['base_id'])

# 4. 결과 확인 및 저장
print("\n--- [DF 데이터셋 분할 결과] ---")
print(df_final.groupby(['split', 'label']).size().unstack(fill_value=0))
print("\n--- [조작 레벨별 분포] ---")
print(df_final.groupby('manipulation_method')['split'].value_counts().unstack(fill_value=0))

os.makedirs(os.path.dirname(SAVE_CSV_PATH), exist_ok=True)
df_final.to_csv(SAVE_CSV_PATH, index=False)
print(f"\n🚀 DF CSV 저장 완료: {SAVE_CSV_PATH}")

📊 발견된 고유 타겟(Base ID) 수: 1000개

--- [DF 데이터셋 분할 결과] ---
label      0      1
split              
test    3000   3000
train  24000  24000
val     3000   3000

--- [조작 레벨별 분포] ---
split                test  train   val
manipulation_method                   
level_3               690   5940   870
level_4               810   6060   630
level_5               690   5940   870
mix_2                 810   6060   630
original             3000  24000  3000

🚀 DF CSV 저장 완료: /content/drive/MyDrive/HECTO/Dataset/CSV/DF_final.csv
